In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("spotify_millsongdata.csv")

In [3]:
df.head(5)

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [4]:
df.tail(5)

,artist,song,link,text
57645,Ziggy Marley,Good Old Days,/z/ziggy+marley/good+old+days_10198588.html,Irie days come on play \r\nLet the angels fly...
57646,Ziggy Marley,Hand To Mouth,/z/ziggy+marley/hand+to+mouth_20531167.html,Power to the workers \r\nMore power \r\nPowe...
57647,Zwan,Come With Me,/z/zwan/come+with+me_20148981.html,all you need \r\nis something i'll believe \...
57648,Zwan,Desire,/z/zwan/desire_20148986.html,northern star \r\nam i frightened \r\nwhere ...
57649,Zwan,Heartsong,/z/zwan/heartsong_20148991.html,come in \r\nmake yourself at home \r\ni'm a ...


In [5]:
df.shape

(57650, 4)

In [6]:
df.isnull().sum()

artist    0
song      0
link      0
text      0
dtype: int64

In [7]:
df =df.sample(5000).drop('link', axis=1).reset_index(drop=True)

In [8]:
df.head(10)

,artist,song,text
0,Christina Aguilera,Fighter,"Well I thought I knew you, thinkin' that you w..."
1,Rihanna,Electricity,Ooooh ohh wooah (can you feel it) \r\nOoooh o...
2,John Prine,Killing The Blues,Leaves were falling ..just like embers \r\nIn...
3,Donna Summer,I Believe In Jesus,Ran into mother Mary down on line \r\nStreet ...
4,Ween,I Play It Off Legit,Where have you been? \r\nI was out with Pat ...
5,Kinks,Get Up,"Here's a song for all the little men, who get ..."
6,Marianne Faithfull,I've Done It Again,"I was on the first ship to Peru, \r\nCharted ..."
7,Hooverphonic,One Way Ride,"Concentrate, you're on the middle \r\nOf this..."
8,Hillsong,I Will Love,"I will love You Lord, my strength \r\nI belon..."
9,Nina Simone,Can't Get Out Of This Mood,All day long before my eyes come little vision...


In [9]:
df['text'][0]

"Well I thought I knew you, thinkin' that you were true  \r\nGuess I, I couldn't trust called your bluff time is up  \r\n'Cause I've had enough  \r\nYou were there by my side, always down for the ride  \r\nBut your joy ride just came down in flames cause your greed sold me out in\r\nshame  \r\n  \r\nAfter all of the stealing and cheating you probably think that  \r\nI hold resentment for you  \r\nBut uh uh, oh no, you're wrong  \r\n'Cause if it wasn't for all that you tried to do, I wouldn't know  \r\nJust how capable I am to pull through  \r\nSo I want to say thank you  \r\nCause it  \r\n  \r\n[Chorus]  \r\n'Cause it makes me that much stronger  \r\nMakes me work a little bit harder  \r\nIt makes me that much wiser  \r\nSo thanks for making me a fighter  \r\nMade me learn a little bit faster  \r\nMade my skin a little bit thicker  \r\nMakes me that much smarter  \r\nSo thanks for making me a fighter  \r\n  \r\nNever saw it coming, all of your backstabbing  \r\nJust so you could cash i

In [10]:
# df = df.sample(5000)

In [11]:
df.shape

(5000, 3)

Text Cleaning/ Text Preprocessing

In [12]:
df['text'] = df['text'].str.lower().replace(r'^\w\s', ' ').replace(r'\n', ' ', regex = True)

In [13]:
import nltk
from nltk.stem.porter import PorterStemmer
nltk.download('punkt')
stemmer = PorterStemmer()

def tokenization(txt):
    tokens = nltk.word_tokenize(txt)
    stemming = [stemmer.stem(w) for w in tokens]
    return " ".join(stemming)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\JAYANSH\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [14]:
df['text'] = df['text'].apply(lambda x: tokenization(x))

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [16]:
tfidvector = TfidfVectorizer(analyzer='word',stop_words='english')
matrix = tfidvector.fit_transform(df['text'])
similarity = cosine_similarity(matrix)

In [17]:
similarity[0]

array([1.        , 0.04881726, 0.09995992, ..., 0.        , 0.02766576,
       0.04008225])

In [18]:
df[df['song'] == 'Crying Over You']

,artist,song,text


In [19]:
def recommendation(song_df):
    idx = df[df['song'] == song_df].index[0]
    distances = sorted(list(enumerate(similarity[idx])),reverse=True,key=lambda x:x[1])
    
    songs = []
    for m_id in distances[1:21]:
        songs.append(df.iloc[m_id[0]].song)
        
    return songs

In [20]:
import pickle
pickle.dump(similarity,open('similarity.pkl','wb'))
pickle.dump(df,open('df.pkl','wb'))